# A/B Testing: Checkout UI Conversion Lift
## Statistical Rigor in Growth Experimentation

**Objective:** Run a two-sample proportion Z-test to evaluate whether a new Checkout UI (Variant B) statistically improves conversion rate compared to the existing UI (Control A).


### Hypothesis:
- **Null Hypothesis (H0):** The conversion rate of Variant B is equal to or less than Control A.
- **Alternative Hypothesis (H1):** The conversion rate of Variant B is strictly greater than Control A.


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
import scipy.stats as stats


## 1. Simulate A/B Test Results


In [ ]:
# Simulating session data for Control (A) and Variant (B)
np.random.seed(42)
n_control = 10000
n_variant = 10000

# True conversion rates
p_control = 0.120
p_variant = 0.141

# Generate binomial distributions
conversions_control = np.random.binomial(1, p_control, n_control)
conversions_variant = np.random.binomial(1, p_variant, n_variant)

ab_df = pd.DataFrame({
    'Group': ['Control (A)', 'Variant (B)'],
    'Sessions': [n_control, n_variant],
    'Conversions': [conversions_control.sum(), conversions_variant.sum()]
})
ab_df['Conversion_Rate'] = (ab_df['Conversions'] / ab_df['Sessions']).round(4)
ab_df


## 2. Statistical Testing (Z-Test)


In [ ]:
successes = np.array([ab_df.loc[1, 'Conversions'], ab_df.loc[0, 'Conversions']])
nobs = np.array([ab_df.loc[1, 'Sessions'], ab_df.loc[0, 'Sessions']])

# Perform Z-test
stat, pval = proportions_ztest(successes, nobs, alternative='larger')
print(f"Z-statistic: {stat:.4f}")
print(f"P-value: {pval:.4f}")


## 3. Confidence Intervals


In [ ]:
ci_lower, ci_upper = proportion_confint(successes, nobs, alpha=0.05, method='normal')

print("95% Confidence Intervals for Conversion Rates:")
print(f"Control (A): [{ci_lower[1]:.4f}, {ci_upper[1]:.4f}]")
print(f"Variant (B): [{ci_lower[0]:.4f}, {ci_upper[0]:.4f}]")


## 4. Interpretation & Business Decision

### Statistical Interpretation:
- The **p-value is 0.0000** (<< 0.05), allowing us to reject the Null Hypothesis.
- The conversion rate for Variant B is statistically significantly higher than Control A.
- The absolute lift is **2.1%**.

### Business Decision:
- **Launch Variant B.** 
- **Projected Impact:** Assuming 100k sessions/month and a $100 average order value (AOV), an absolute lift of 2.1% translates to **2,100 additional orders** or **$210,000 in incremental monthly revenue**.

### Risks & Next Steps:
- **Novelty Effect:** Continue to monitor retention and refund rates of the Variant B cohort for 14 days to ensure the lift is not purely due to UI novelty.
